In [1]:
def load_data_set():
    # 달마시안 애호가 전자 게시판에서 가져온 문서 집합을 토큰화 한 것
    posting_list = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                   ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                   ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                   ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                   ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                   ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]

    # 1: 폭력적임, 0: 폭력적이지 않음
    # 텍스트에 사람이 직접 분류 항목 표시를 붙이고, 폭력적인 게시물을 자동적으로 감지하는 프로그램을 훈련하는데 사용한다.
    class_vec = [0,1,0,1,0,1] 
    return posting_list, class_vec

# 모든 문서에 있는 모든 유일한 단어 목록을 생성한다.
def create_vocab_list(data_set):
    # 비어있는 집합 생성
    vocab_set = set([]) 
    for document in data_set:
        # 연산자 | 는 두 개의 집합 유형의 변수를 합치는 데 사용하는 연산자.
        # 두 개의 집합 통합 생성
        # ex) {'a', 'b', 'c'} | {'c', 'd', 'e'} 
        #     -> {'a', 'b', 'c', 'd', 'e'} 
        vocab_set = vocab_set | set(document)
    return list(vocab_set)

# 주어진 문서 내에 어휘 목록에 있는 단어가 존재하는지 아닌지를 표현하기 위해 어휘 목록, 문서, 
# 1과 0의 출력 벡터를 사용한다.
def set_of_words_2_vec(vocab_list, input_set):
    # 모두 0인 벡터 생성
    return_vec = [0] * len(vocab_list) 
    for word in input_set:
        if word in vocab_list:
            return_vec[vocab_list.index(word)] = 1
        else:
            print ("the word: %s is not in my Vocablulary!" % word)
    return return_vec

# Test
list_o_posts, list_classes = load_data_set()
my_vocab_list = create_vocab_list(list_o_posts)
my_vocab_list
set_of_words_2_vec(my_vocab_list, list_o_posts[0])


[1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1]

In [2]:
from numpy import *

def train(train_matrix, train_category):
    # 문서의 수
    num_train_docs = len(train_matrix)
    
    # 한문서의 최대 단어수
    num_words = len(train_matrix[0])
    
    # 폭력적인 문서의 확률 계산
    # 1의 값이 폭력적이기 때문에 train_category를 합하면 폭력적인 문서의 수가 나온다.
    # 폭력적인 문서 수 / 전체 문서 수
    # 현재는 분류 항목이 두개 이기때문에 가능 / 분류 항목이 2개 이상이면 이부분을 수정해야 한다.
    p_abusive = sum(train_category) / float(num_train_docs)
    
    # 확률 초기화
    # zerors: num_words(인자값) 수 만큼 0. 배열 만듬
    p_0_num = ones(num_words) 
    p_1_num = ones(num_words)
    p_0_denom = 2.0
    p_1_denom = 2.0
    
    # 벡터 추가 
    for i in range(num_train_docs):
        if train_category[i] == 1:
            p_1_num += train_matrix[i]
            p_1_denom += sum(train_matrix[i])
        else:
            p_0_num += train_matrix[i]
            p_0_denom += sum(train_matrix[i])

    # 원소 나누기
    p_1_vect = log(p_1_num / p_1_denom)
    p_0_vect = log(p_0_num / p_1_denom)
    return p_0_vect, p_1_vect, p_abusive

# Test
list_o_posts, list_classes = load_data_set()
my_vocab_list = create_vocab_list(list_o_posts)
train_matrix = []
# 리스트 단어 벡터로 채운다
for post_in_doc in list_o_posts:
    train_matrix.append(set_of_words_2_vec(my_vocab_list, post_in_doc))
p_0_v, p_1_v, p_ab = train(train_matrix, list_classes)

In [3]:
"""
vec_2_classify: 분류를 위한 벡터
p_0_v, p_1_v, p_ab: train()에서 계산된 확률
"""
def classify(vec_2_classify, p_0_vec, p_1_vec, p_class_1):
    # 원소 곱하기
    # 방식은 두 벡터의 첫 번째 원소들을 곱한 뒤, 두 번째 원소들을 곱하고, 이런식으로 계속해서 끝까지 곱해 가는 방식이다. 
    # 그런다음, 어휘집에 있는 모든 단어들에 대한 값을 더하고, 분류 항목의 로그 확률에 더한다.
    # 
    p1 = sum(vec_2_classify * p_1_vec) + log(p_class_1)
    p0 = sum(vec_2_classify * p_0_vec) + log(1.0 - p_class_1)
    
    if p1 > p0:
        return 1
    else:
        return 0

def testing():
    list_o_posts, list_classes = load_data_set()
    my_vocab_list = create_vocab_list(list_o_posts)
    
    train_matrix = []
    
    for post_in_doc in list_o_posts:
        train_matrix.append(set_of_words_2_vec(my_vocab_list, post_in_doc))
    
    p_0_v, p_1_v, p_ab = train(array(train_matrix), array(list_classes))
    
    test_entry = ['love', 'my', 'dalmation']
    this_doc = array(set_of_words_2_vec(my_vocab_list, test_entry))
    
    print (test_entry, 'classfied as: ', classify(this_doc, p_0_v, p_1_v, p_ab))
    
    test_entry = ['stupid', 'garbage']
    this_doc = array(set_of_words_2_vec(my_vocab_list, test_entry))
    
    print (test_entry, 'classified ad: ', classify(this_doc, p_0_v, p_1_v, p_ab))

#Test

testing()

['love', 'my', 'dalmation'] classfied as:  0
['stupid', 'garbage'] classified ad:  1


In [4]:
# 중복단어 모델 
def bag_of_words_2_vec_MN(vocab_list, input_set):
    return_vec = [0] * len(vocab_list)
    for word in input_set:
        if word in vocab_list:
            return_vec[vocab_list.index(word)] += 1
    return return_vec

# Test
my_sent = ' This book is the best book on Python or M.L. I have ever laid eyes upon.'
my_sent.split()

['This',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'Python',
 'or',
 'M.L.',
 'I',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon.']

In [ ]:
import re

# 구두점 제거
reg_ex = re.compile('\\W*')
list_of_tokens = reg_ex.split(my_sent)

# 빈 문자열 제거 및 소문자 변환 (문자변환: lower, upper)

[tok.lower() for tok in list_of_tokens if len(tok) > 0]

In [ ]:
email_text = open('ham/6.txt').read()
list_of_tokens = reg_ex.split(email_text)
list_of_tokens

In [5]:
# 큰 문자열을 처리하며, 문자열 리스트로 텍스트를 구문 분석한다.
# 길이가 2개 이하인 단어는 탈락시키며, 모든 단어를 소문자로 변환한다.
def text_parse(big_string):
    import re
    list_of_tokens = re.split(r'\W*', big_string)
    return [tok.lower() for tok in list_of_tokens if len(tok) > 2]

# 나이브 베이스 스팸 분류기를 자동화한다.
def spam_test():
    doc_list = []
    class_list = []
    full_text = []
    
    # 스팸과 햄의 텍스트 파일을 단어 리스트로 불러온다.
    for i in range(1, 26):
        word_list = text_parse(open('spam/%d.txt' % i).read())
        doc_list.append(word_list)
        full_text.extend(word_list)
        class_list.append(1)
        
        word_list = text_parse(open('ham/%d.txt' % i).read())
        doc_list.append(word_list)
        full_text.extend(word_list)
        class_list.append(0)
    
    # 검사 집합과 훈련집합을 생성한다.
    vocab_list = create_vocab_list(doc_list)
    training_set = list(range(50))
    test_set = []

    # 10개를 랜덤으로 생성해서 훈련용 데이터와 검사용데이터로 나눈다.
    for i in range(10):
        rand_index = int(random.uniform(0, len(training_set)))
        test_set.append(training_set[rand_index])
        print ('삭제 index: %d' % rand_index)
        del(training_set[rand_index])
    
    train_matrix = []
    train_classes = []
    # 검사 집합에 있는 모든 아이템을 반복하며, 각 이메일과 어휘집에 있는 단어들로부터 set_of_words_2_vec를 사용하여 단어 벡터를 생성한다.
    for doc_index in training_set:
        train_matrix.append(set_of_words_2_vec(vocab_list, doc_list[doc_index]))
        train_classes.append(class_list[doc_index])

    # 단어들은 분류에 필요한 확률을 계산하기위해 train 데이터를 사용한다.
    p_0_v, p_1_v, p_spam = train(array(train_matrix), array(train_classes))
    error_count = 0
    
    # 검사 집합을 반복하고 검사 집합 내에서 각 이메일을 분류한다. 
    # 분류가 제데로 되지 않았다면 오류 개수를 증가한다.
    for doc_index in test_set:
        word_vector = set_of_words_2_vec(vocab_list, doc_list[doc_index])
        if classify(array(word_vector), p_0_v, p_1_v, p_spam) != class_list[doc_index]:
            error_count += 1
    print ('The error rate is: ', float(error_count) / len(test_set))    
# Test
spam_test()        

삭제 index: 15
삭제 index: 48
삭제 index: 28
삭제 index: 8
삭제 index: 27
삭제 index: 41
삭제 index: 29
삭제 index: 29
삭제 index: 10
삭제 index: 21
The error rate is:  0.0


In [ ]:
# 나이브 베이스를 사용하여 개인 광고에 포함된 지역 특색 도출하기 
# 다른 지역에 있는 사람들은 다른 단어를 사용한다는 것을 확인할 수 있을 것이다.

import feedparser
ny = feedparser.parse('http://newyork.craigslist.org/stp/index.rss')
len(ny['entries'])

In [4]:
# 어휘집에 있는 모든 단어를 훑으면서 단어가 텍스트에 얼마나 많이 출현하는지 센다.
# 딕셔너리에는 빈도가 가장 높은 것부터 가장 낮은 순으로 정렬되어 있으며, 이 중 상위 100개의 단어를 반환한다.
def calc_most_freq(vocab_list, full_text):
    import operator
    freq_dict = {}
    # 발생 빈도 계산
    for token in vocab_list:
        freq_dict[token] = full_text.count(token)
    
    sorted_freq = sorted(freq_dict.items(), key=operator.itemgetter(1), reverse=True)
    # 빈도를 바꾸어 계산해보자
    return sorted_freq[:30]

"""
feed는 외부에서 블러와야 한다.
spamTest와 비슷하다. 
다른것은 파일 대신 피드를 처리한다는 것과 100개의 단어를 얻어 이 단어들을 제거하기 위해 cal_most_freq를 호출한다는 것이다.
"""
def local_words(feed1, feed2):
    import feedparser
    doc_list = []
    class_list = []
    full_text = []
    min_len = min(len(feed1['entries']), len(feed2['entries']))
    for i in range(min_len):
        # 한 번에 하나의 피드 처리
        word_list = text_parse(feed1['entries'][i]['summary'])
        doc_list.append(word_list)
        full_text.extend(word_list)
        class_list.append(1)
        
        word_list = text_parse(feed2['entries'][i]['summary'])
        doc_list.append(word_list)
        full_text.extend(word_list)
        class_list.append(0)
    

    vocab_list = create_vocab_list(doc_list)
    top_30_words = calc_most_freq(vocab_list, full_text)
    
    # 발생 빈도가 가장 많은 단어 삭제
    """
    게시물에 있는 단어중 상위 30개의 단어가 사용된 모든 단어의 30% 정도를 차지한다는 것이다.
    즉, 사용된 전체 단어 중 몇 개의 단어가 텍스트 문서의 많은 부분을 차지한다는 것이다.
    """
    for pair_w in top_30_words:
        if pair_w[0] in vocab_list:
            vocab_list.remove(pair_w[0])

    training_set = list(range(2 * min_len))
    test_set = []
    
    for i in range(20):
        rand_index = int(random.uniform(0, len(training_set)))
        test_set.append(training_set[rand_index])
        del(training_set[rand_index])
    
    train_matrix = []
    train_classes = []
    
    for doc_index in training_set:
        train_matrix.append(bag_of_words_2_vec_MN(vocab_list, doc_list[doc_index]))
        train_classes.append(class_list[doc_index])
    
    p_0_v, p_1_v, p_sapm = train(array(train_matrix), array(train_classes))
    error_count = 0
    
    for doc_index in test_set:
        word_vector = bag_of_words_2_vec_MN(vocab_list, doc_list[doc_index])
        if classify(array(word_vector), p_0_v, p_1_v, p_spam) != class_list[doc_index]:
            error_count += 1
    
    print ('The error rate is: ', float(error_count) / len(test_set))
    return vocab_list, p_0_v, p_1_v
           
# Test

ny = feedparser.parse('http://newyork.craigslist.org/stp/index/rss')
sf = feedparser.parse('http://sfbay.craigslist.org/stp/index.rss')
vocab_list, p_SF, p_NY = local_words(ny, sf)

NameError: name 'feedparser' is not defined

In [166]:


def get_top_words(ny, sf):
    import operator
    
    # 나이스베이스 분류기를 훈련하고 검사한다. 이때 사용된 확률은 반환된다.
    vocab_list, p_0_v, p_1_v = local_words(ny, sf)
    
    # 그런다음 두개의 리스트를 생성한다.
    top_NY = []
    top_SF = []
    
    # 리스트 내부는 튜플 형태로 저장한다.
    # 상위 X개의 단어를 반환하는 것보다 특정 임계 값 이상인 단어를 모두 반환하는 것이 좋다.
    for i in range(len(p_0_v)):
        if p_0_v[i] > -6.0:
            top_SF.append((vocab_list[i], p_0_v[i]))
        if p_1_v[i] > -6.0:
            top_NY.append((vocab_list[i], p_1_v[i]))
    
    sorted_SF = sorted(top_SF, key=lambda pair: pair[1], reverse=True)
    
    print ("SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**")
    
    for item in sorted_SF:
        print (item[0])
    
    sorted_NY = sorted(top_NY, key=lambda pair: pair[1], reverse=True)
    
    print ("NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**")
    for item in sorted_NY:
        print (item[0])
        
# Test

get_top_words(ny, sf)

# 유의사항
# 출력된 단어에 중지 단어가 많이 나타난다는 것이다. 미리 선정한 중지 단어를 제거하고,
# 결과가 어떻게 변화하는지 확인하는 것도 흥미로울 것이다.
        

The error rate is:  0.45
SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**
text
need
fun
wanna
want
are
from
share
let
contact
what
would
hit
send
fit
single
lets
drink
our
title
various
nice
guy
new
email
woman
never
guys
movie
home
death
info
interested
exploring
gym
show
good
little
see
married
lady
meet
try
outdoors
female
actually
rut
snuggling
quirk
kind
prints
notes
buy
still
girls
hear
bored
marriage
bottom
showcontact
class
blaze
step
sleep
haha
year
bit
hot
athletic
vacation
straight
guess
know
general
variety
pics
hard
again
stuck
wide
blankets
vajayjays
under
african
curious
been
mind
flirty
change
trade
before
preference
answer
compare
nothing
loves
click
bed
watch
tell
one
old
bare
could
through
realm
kik
while
stats
longer
animals
smartass
even
open
art
meal
tonight
hello
threesome
snuggled
might
friends
chris
sfo
lookin
hope
post
buddy
hey
male
games
whether
believes
perhaps
host
fantasies
person
please
has
stp
upbeat
extra
work
playing
eventually
motivate
own
possibly
t

/Users/hwayo/.pyenv/versions/machinelearning/lib/python3.4/site-packages/IPython/kernel/__main__.py:33: RuntimeWarning: divide by zero encountered in log
/Users/hwayo/.pyenv/versions/machinelearning/lib/python3.4/site-packages/IPython/kernel/__main__.py:34: RuntimeWarning: divide by zero encountered in log
/Users/hwayo/.pyenv/versions/machinelearning/lib/python3.4/site-packages/IPython/kernel/__main__.py:10: RuntimeWarning: invalid value encountered in multiply
/Users/hwayo/.pyenv/versions/machinelearning/lib/python3.4/site-packages/IPython/kernel/__main__.py:11: RuntimeWarning: invalid value encountered in multiply


In [ ]:
"""
베이지안 확률과 베이스 규칙은 우리에게 알려진 값으로부터 알려지지 않은 것의 확률을 추정하는 방법을 제공한다.

데이터 속성 간의 관계가 조건부 독립이라고 가정함으로써 필요한 데이터의 양을 많이 줄일 수 있다.

우리가 세운 가정은 문서에서 한 단어의 확률이 다른 어떠한 단어에도 의존하지 않는다는 것이다.
(이것은 나이브 베이스로 알려진 방법이다.(
잘못된 가정에도 불구하고 나이브 베이스는 분류에 효과적이다.

언더플로우 문제 계산을 할 때 확률에 로그를 사용함으로써 해결할 수 있다.
문서 분류 처리에 있어서 중복 단어 모델은 집합 단어 모델을 개선한 것이다.
중지 단어를 삭제하는 것처럼 개선하는 방법도 여러가지가 있다.
당신은 토큰을 최적화 하는데 많은 시간을 보내게 될 것이다.

"""